#### Использованные библиотеки

In [ ]:
!pip install -r requirements.txt

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F

import warnings
warnings.filterwarnings("ignore")

torch.set_float32_matmul_precision('medium')
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

### Генерация датасета

Прежде чем лететь ставить эксперименты, нужно разобраться с тем, как же всё-таки генерировать новые таски из имеющегося датасета. Учитывая, что их число может доходить до $2^{24}$, это не так просто

1. Сперва нужны сами датасеты. Для примера возьмём MNIST, для него же нужно будет собрать трансформации.
   Они здесь написано костыльно, просто потому что вдруг что-то сломается, но суть такая: берём картинку,
   плодим каналы, поскольку она чёрно-белая, ресайзим, транспонируем каналы, чтобы потом её нормально распрямить, кастим во флоты. Трансформация ниже по факту оставляет форму картинки, как есть, но понятно, что это легко поменять

In [4]:
from torchvision.transforms import v2
from torchvision.datasets import MNIST

g_transform = v2.Compose([
    v2.Lambda(lambda x: x.repeat(3, 1, 1, 1)),
    v2.Resize(28),
    v2.Lambda(lambda x: x.transpose(0, 1)),
    v2.Grayscale(),
    v2.Lambda(lambda x: x.flatten(1)),
    v2.ConvertImageDtype(torch.float),
])

У меня все датасеты заворачиваются в мой класс, который просто чуть удобнее применяет трансформации и делает общий интерфейс для всех датасетов

In [19]:
from datagen import SubLoader

mnist_train = SubLoader(MNIST(
    './datasets', train=True, download=True, transform=g_transform
)) 
mnist_train

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./datasets
    Split: Train
    StandardTransform
Transform: Compose(
                 Lambda(<lambda>, types=['object'])
                 Resize(size=[28], interpolation=InterpolationMode.BILINEAR, antialias=warn)
                 Lambda(<lambda>, types=['object'])
                 Grayscale(num_output_channels=1)
                 Lambda(<lambda>, types=['object'])
                 ConvertImageDtype()
           )

2. Следующий шаг это применить какую-нибудь аугментацию и посмотреть, что с ним станет

Напомню, что в статье трансформацией датасета считается
<img src="https://media.discordapp.net/attachments/674191702906503199/1194320451283980349/image.png?ex=65afec98&is=659d7798&hm=c9b27e967024dc88b819a2ccb80aac09d14fd49db607efcc4feb1aa202279037&=&format=webp&quality=lossless&width=749&height=457" width="500px">

$D_{\text{orig}} = \{x_i, y_i\}_{i=1}^{N_D}$ - старый датасет \
$D = \{A_{n}x_i, p_n(y)_i\}_{(i=1, \ n=1)}^{(N_D, \ N_n)}$ - новый датасет \
$A \in \mathbb{R}^{N_x}, A_{ij} \in \mathcal{N}(0,\,\frac{1}{N_x})$ - линейный проектор \
$p(y) \in S_{N_y}$ - перестановка на множестве таргетов \
$N_D$ - размерность всего датасета, например 60к картинок из MNIST \
$N_n$ - число новых тасок \
$N_x$ - размерность входных данных, в нашем случае картинок \
$N_y$ - число классов таргета \
$Ax_i$ - проекция $i$-го объекта \
$p(y)_i$ - $i$-ый таргет после перестановки. Важно(!) я не делаю onehot специально, потому что использую кросс-энтропию

Матриц и перестановок должно быть столько, сколько новых тасок мы хотим нагенерить, потом их склеиваем в один большой датасет и обучаем что-нибудь

In [72]:
from datagen import TaskAugmentor

augmentor = TaskAugmentor(
    n_tasks=4, draw_sequence=False, random_state=69, device="cpu"
)
# не нормируем, иначе не сравним
augmented_mnist = augmentor.transform(mnist_train, normalize=False)
print(f"{mnist_train.data.shape} -> {augmented_mnist.data.shape}")
print(f"{mnist_train.targets.shape} -> {augmented_mnist.targets.shape}")

torch.Size([60000, 28, 28]) -> torch.Size([240000, 784])
torch.Size([60000]) -> torch.Size([240000])


Все они генерируют строго те же самые трансформации, если зафксировать сид. Сделано это по-колхозному, но мне главное, что работает. Проверим, что проекция и перестановка действительно применяются к данным

In [61]:
augmentor._generation_seed

tensor([1272380470, 1724767435, 3474919369, 2559044203])

In [33]:
from datagen import LinearProjection, TargetPermutation

projection = LinearProjection(28*28*1, random_state=1272380470)
permutation = TargetPermutation(10, random_state=1272380470)

In [82]:
# всё, как надо по статье
m = projection.transformation_matrix
m.mean(), m.std(), 1/784

(tensor(1.1410e-06), tensor(0.0013), 0.0012755102040816326)

Возьмём картинку и таргет. Автотрансформ от торчивжна там отключён, потому что он применяется
лишь к одному объекту, а не ко всем сразу, как этого хочется. Надо сделать вручную

In [73]:
image, target = mnist_train.transform(mnist_train[0])
image.shape, target

(torch.Size([1, 784]), tensor(5))

In [85]:
(torch.allclose(projection(image), augmented_mnist.data[0]),
 torch.allclose(permutation(target), augmented_mnist.targets[0]))

(True, True)

Получаем то, что нужно \
Для последовательностей всё чуть-чуть хитрее. Они все сэмплируются по умолчанию, просто потому что иначе будет слишком большая последовательность. Идея статьи в том, чтобы подавать их все сразу в один аттеншн, а 60к картинок туда не влезут

In [93]:
augmentor = TaskAugmentor(
    n_tasks=4, draw_sequence=True, random_state=69, device="cpu"
)
# не нормируем, иначе не сравним
augmented_mnist_seq = augmentor.transform(mnist_train, normalize=False)

In [94]:
augmented_mnist_seq.data.shape, augmented_mnist_seq.targets.shape

(torch.Size([4, 100, 794]), torch.Size([4, 100]))

Тут специально тест написать будет посложнее, поэтому я не хочу этим заниматься, но по логике кода всё должно быть верно. Также, как видно по размерности, к инпуту приклеиваются onehot-таргеты, сдвинутые вправо, как на картинке, чтобы учиться предсказывать по произвольному префиксу тоже \
<img src="https://media.discordapp.net/attachments/674191702906503199/1194319105839333478/image.png?ex=65afeb58&is=659d7658&hm=98878da6184eca24f9dedb85a9be58df3bc58ff490f54016f9f38111a91e7ef4&=&format=webp&quality=lossless" width="500px"> \
Это обеспечивается через
`python
prev_targets[:, 1:] = F.one_hot(new_dataset.targets[:, :-1], 10)
new_dataset.data = torch.cat([new_dataset.data, prev_targets], dim=-1)
`

Вот и все аугментации. Идейно ничего сложного нет, но с точки зрения вычислений там есть, что пооптимизировать, хотя это можно глянуть в `datagen.py`. Пока бенчмарк такой - $2^{16}$ тасок генерируются за 2 минуты, но по памяти бьют сильно, потому что сохраняются локально, занимают почти 40Гб, как минимум вот эту штуку было бы хорошо переделать, чтобы читать лоадером из файла